##Import libraries

In [0]:
%pip install lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/3.6 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/3.6 MB 1.2 MB/s eta 0:00:03
   ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/3.6 MB 872.7 kB/s eta 0:00:05
   ━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/3.6 MB 1.1 MB/s eta 0:00:04
   ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.2/3.6 MB 1.7 MB/s eta 0:00:02
   ━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.4/3.6 MB 2.1 MB/s eta 0:00:02
   ━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.5/3.6 MB 2.8 MB/s eta 0:00:02
   ━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/3.6 MB 4.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━ 1.5/3.6 MB 5.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━ 2.2/3.6 MB 7.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 3.6/3.6 MB 10.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 3.6/3.6 MB 10.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 9.2 MB/s eta 0:00:00
Note: you may n

In [0]:
import random

import joblib

import os

import pandas as pd
import numpy as np
import pyspark.pandas as ps

import lightgbm as lgb

from pyspark.sql.functions import *
from pyspark.sql import Window

from pyspark.sql.functions import *
from pyspark.sql import Window
from pyspark.sql.types import *

In [0]:
orig_table = 'bfl_std_lake.sme_coe.dj_plcs_offers_latest_bureau'

id = 'id'

etb_table  = 'bfl_std_lake.sme_coe.zm_plcs_offers_latest_bureau_etb_vars'
bvar_table = 'bfl_std_lake.sme_coe.dj_plcs_offers_latest_bureau_new_vars'

var_table  = 'bfl_std_lake.sme_coe.zm_plcs_offers_latest_bureau_all_vars'
fin_table  = 'bfl_std_lake.sme_coe.zm_plcs_offers_latest_bureau_plcs_prop'

In [0]:
%py
spark.sql(f"""create or replace table {var_table}
as
select
A.*,
B.* except (B.{id}),
C.* except (C.{id})
from
{orig_table} a
left join {etb_table} b on a.{id} = b.{id}
left join {bvar_table} c on a.{id} = c.{id}""")

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
import warnings

warnings.filterwarnings('ignore')

In [0]:
df = spark.read.table(var_table)

final_fts = ['bureaucustomerid','months_latest_loan','current_age','bureau_vintage','etb_total_loans','etb_total_pos','avg_irr','cnt_total_loan','max_overdue','months_latest_unsec','avg_amt','months_oldest_unsec','ever_3in1_flag','cnt_total_l12m','cnt_cat6_loan','dpd_1plus_3m_bf_date','dpd_30plus_24m_bf_date','enq_pl_l12m','tier']

df = df.select(*final_fts)

##Handling null values

In [0]:
mean_age = int(np.rint(df.select(mean(col('current_age'))).collect()[0].__getitem__('avg(current_age)')))

df = df.fillna('OTHERS', subset='TIER')
df = df.fillna(mean_age, subset='current_age')
df = df.fillna(0, subset=None)

pd_df = df.pandas_api()

pd_df['bureaucustomerid'] = pd_df['bureaucustomerid'].astype(str)

##Encoding

In [0]:
pd_df = ps.get_dummies(pd_df,columns=['tier'])

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
AttributeError: 'NoneType' object has no attribute 'rint'

The above exception was the direct cause of the following exception:

TypeError                                 Traceback (most recent call last)
File <command-1255184448117989>, line 7
      3 final_fts = ['bureaucustomerid','months_latest_loan','current_age','bureau_vintage','etb_total_loans','etb_total_pos','avg_irr','cnt_total_loan','max_overdue','months_latest_unsec','avg_amt','months_oldest_unsec','ever_3in1_flag','cnt_total_l12m','cnt_cat6_loan','dpd_1plus_3m_bf_date','dpd_30plus_24m_bf_date','enq_pl_l12m','tier']
      5 df = df.select(*final_fts)
----> 7 mean_age = int(np.rint(df.select(mean(col('current_age'))).collect()[0].__getitem__('avg(current_age)')))
      9 df = df.fillna('OTHERS', subset='TIER')
     10 df = df.fillna(mean_age, subset='current_age')

TypeError

In [0]:
obj_list = []

for i,j in zip(pd_df.columns,pd_df.dtypes):
  if j=='object' and i not in ['bureaucustomerid']:
    obj_list.append(i)

for i in obj_list:
  pd_df[i] = pd_df[i].astype(float)

In [0]:
import mlflow
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType

##Loading of Production Model

In [0]:
# Configuration
MODEL_NAME = "PLCS_Propensity_Model"
# This loads the model version currently designated as 'Production' in the MLflow Registry
MODEL_URI = f"models:/{MODEL_NAME}/Production"

In [0]:
# Identify the feature columns (must match the training features)
feature_cols = [c for c in df.columns if c not in ['bureaucustomerid']]

In [0]:
# Prepare data for the Spark UDF: Create a struct of all features
df_for_inference = df.withColumn(
    "features_input_struct", 
    struct(*feature_cols)
)

In [0]:
# Load the model as a scalable Spark UDF
predict_udf = mlflow.pyfunc.spark_udf(
    spark, 
    model_uri=MODEL_URI, 
    result_type=DoubleType() 
)

In [0]:
# Apply the UDF for prediction
df_with_prob = df_for_inference.withColumn(
    "lgb_prob", 
    predict_udf(col("features_input_struct"))
).drop("features_input_struct", *feature_cols) # Drop the input struct and features to keep only ID and probability

In [0]:
w = Window.partitionBy('bureaucustomerid').orderBy(col('lgb_prob').desc())

In [0]:
df = df.withColumn('PLCS_PROPENSITY',when(col('lgb_prob').isNull(),lit(None))
                                        .when(col('lgb_prob') >= 0.90,'P01')
                                        .when(col('lgb_prob') >= 0.85,'P02')
                                        .when(col('lgb_prob') >= 0.80,'P03')
                                        .when(col('lgb_prob') >= 0.75,'P04')
                                        .when(col('lgb_prob') >= 0.70,'P05')
                                        .when(col('lgb_prob') >= 0.65,'P06')
                                        .when(col('lgb_prob') >= 0.60,'P07')
                                        .when(col('lgb_prob') >= 0.55,'P08')
                                        .when(col('lgb_prob') >= 0.50,'P09')
                                        .when(col('lgb_prob') >= 0.45,'P10')
                                        .when(col('lgb_prob') >= 0.40,'P11')
                                        .when(col('lgb_prob') >= 0.35,'P12')
                                        .when(col('lgb_prob') >= 0.30,'P13')
                                        .when(col('lgb_prob') >= 0.25,'P14')
                                        .when(col('lgb_prob') >= 0.20,'P15')
                                        .when(col('lgb_prob') >= 0.15,'P16')
                                        .when(col('lgb_prob') >= 0.10,'P17')
                                        .when(col('lgb_prob') >= 0.05,'P18')
                                        .when(col('lgb_prob') >= 0.025,'P19')
                                        .otherwise('P20')).withColumn('rnk',row_number().over(w)).filter(col('rnk')==1).drop('rnk')

In [0]:
orig_df = spark.read.table(orig_table)

In [0]:
final_table = orig_df.join(df,'bureaucustomerid','left')

In [0]:
spark.sql(f'drop table if exists {fin_table}')

In [0]:
df.write.saveAsTable(fin_table)